# Create Heatmaps for significant ATR Signaling genes

Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [23]:
prot_enr.res2d.iloc[[7]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
7,ATR signaling pathway Homo sapiens 8991cbac-61...,14/39,0.000248,0.006475,0,0,2.749708,22.830131,RFC5;RFC3;TIPIN;RFC4;MCM7;RFC2;PLK1;RPA1;FANCD...,NCI-Nature_2016


# Step 2: Get the list of significant genes 

In [24]:
i = 7
trans = prot_enr.res2d.Genes[i]
genes = trans.split(';')
print(prot_enr.res2d.Term[i])
print('total genes:',len(genes))

ATR signaling pathway Homo sapiens 8991cbac-618b-11e5-8ac5-06603eb7f303
total genes: 14


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [32]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

14

In [33]:
# Only include p-values < a
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [28]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 600)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [29]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [34]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['RFC5', 'MCM2', 'MCM7', 'RFC2', 'TIPIN', 'RFC4', 'TOPBP1', 'CHEK1',
       'PLK1'], dtype=object)

In [35]:
# Only include p-values < a
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [38]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=500)

Loading BokehJS ...

# Sig in multiple cancers

In [39]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [41]:
enr2.res2d.iloc[[4]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
4,ATR signaling pathway Homo sapiens 8991cbac-61...,6/39,0.000029,0.001192,0,0,9.893643,103.533042,RFC3;RFC4;MCM7;RFC2;TOPBP1;MCM2,NCI-Nature_2016


In [43]:
i = 4
trans2 = enr2.res2d.Genes[i]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[i])
print('total genes:',len(genes_mult))

ATR signaling pathway Homo sapiens 8991cbac-618b-11e5-8ac5-06603eb7f303
total genes: 6


In [44]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['MCM2', 'MCM7', 'RFC3', 'RFC2', 'RFC4', 'TOPBP1'], dtype=object)

In [45]:
# Only include p-values < 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= 0.01]

In [48]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=500)

Loading BokehJS ...

In [49]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM2', 'MCM7', 'RFC2', 'RFC4', 'TOPBP1'], dtype=object)

In [50]:
# Only include p-values < 0.01
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= 0.01]

In [52]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=400)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [53]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

['MCM2', 'MCM7', 'RFC2', 'RFC4', 'TOPBP1']


In [54]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,ATR signaling pathway Homo sapiens 8991cbac-61...,5/39,2.159149e-14,4.512621e-12,0,0,512.820513,16136.654942,RFC4;MCM7;RFC2;TOPBP1;MCM2,NCI-Nature_2016
1,Fanconi anemia pathway Homo sapiens 6befb873-6...,3/47,1.212150e-07,1.266696e-05,0,0,255.319149,4066.136239,RFC4;RFC2;TOPBP1,NCI-Nature_2016
2,BARD1 signaling events Homo sapiens 75b04491-6...,1/29,7.229642e-03,5.036650e-01,0,0,137.931034,679.940112,TOPBP1,NCI-Nature_2016
3,E2F transcription factor network Homo sapiens ...,1/72,1.787250e-02,9.338380e-01,0,0,55.555556,223.582899,TOPBP1,NCI-Nature_2016


In [21]:
# Look at p-values 
certain_gene = 'SYK'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
499,SYK,0.000320,-0.697550,Gbm
11499,SYK,0.001919,-0.225323,Hnscc
23264,SYK,0.021461,-0.489000,Luad
40690,SYK,0.675254,0.222300,Lscc
53912,SYK,0.905056,0.020400,Brca
54758,SYK,0.003901,0.405618,Ov
67986,SYK,0.085643,0.079000,En
82040,SYK,0.775873,0.034700,Colon
